In [1]:
library(ggplot2)
library(caret)
library(dplyr)
library(randomForest)

Loading required package: lattice

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin



In [2]:
set.seed(1)

In [3]:
jRoadType <- c("都市間高速", "都市高速", "有料道路", "国道", "県道", "主要地方道", "一般道1、一般道2、一般道3", "その他")
jHighSpeeds <- c("都市間高速", "都市高速", "有料道路")

In [4]:
# 利用する特徴量を列挙
valuables <- c("RoadType","CurveAverage","Speed", "Curve100", "Curve150","MaxSpeed","RiskFactor","Curve","DistSignal","Pitch","AheadDistance","AverageVelocity","TimeToCollision","AccelerationSpeed", "Engine", "SteeringAngle", "TimeHeadway", "Jerk", "LaneCount", "DiffAvgSpeed", "EmptinessOfRoad", "RoadFactor")

In [5]:
printf <- function(...) cat(sprintf(...))

In [6]:
# CV value を計算
# dfx: 使用するデータ
# verbose: ログを出力するかどうか
CV <- function(dfx, verbose=FALSE) {        
    # k=5 束の fold を作成
    folds <- createFolds(dfx$flag, k=5)
    
    # 各 fold における結果を格納する変数
    count <- 1
    errs <- c()       # CV 値
    reds <- c()     # 長時間平均 (Red だと予測して実際に Red だったもの)
    blues <- c()    # 長時間平均 (Blue だと予測して実際に Blue だったもの)
    shorts <- c()  # 短時間平均
    for (ids in folds) {
        train <- dfx[-ids, c(valuables, "flag")]                                            # 予測に使う特徴量だけ抜き出す (AccelOrBrake を抜く)
        test <- dfx[ids, ]
        fit <- randomForest(flag ~ ., data=train, mtry=3, ntree=300)   # Random Forest で学習
        p <- predict(fit, newdata=test)                                                      # test データを使って予測
        
        # 予測した結果を用いて短期、長期平均を計算
        if (verbose) {
            printf("Fold%d\n", count)
            result <- correctVsPredict(test, p, verbose)
            shorts <- c(shorts, shortErr(test, p, verbose))
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
            count <- count + 1
            printf("\n")
        } else {
            result <- correctVsPredict(test, p, verbose)
            shorts <- c(shorts, shortErr(test, p, verbose))
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
        }
        
        # CV 値を計算
        nerr <- sum(p != test$flag)
        errs <- c(errs, nerr / nrow(test))
    }
    
    c(mean(errs), mean(shorts), mean(reds), mean(blues))
}

In [7]:
# 短時間平均を計算する
shortErr <- function(test, predictedFlags, verbose=FALSE) {
    # Red と予測したテストのうち、Red かつ Accel を取り出す
    predictedRedRowsInTrue <- nrow(test[test$flag == "Red" & test$AccelOrBrake == "Accel" & predictedFlags == "Red", ])
    # テストのうち Red かつ Accel を取り出す
    nRedAccel <- nrow(test[test$flag == "Red" & test$AccelOrBrake == "Accel", ])
    
    if (verbose) {
      printf("Short: predictedRedAInTrueA/trueRedA = %d/%d = %f\n", predictedRedRowsInTrue, nRedAccel, predictedRedRowsInTrue/nRedAccel)
      # print(predictedRedRowsInTrue %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
    }
    
    predictedRedRowsInTrue/nRedAccel
}

In [8]:
# 長時間平均を計算する
correctVsPredict <- function(test, predictedFlags, verbose=FALSE) {    
    # for Red
    predictedRedRows <- test[predictedFlags == "Red", ]
    nCorrectReds <- sum(predictedRedRows$flag == 'Red')
    nPredictedReds <- nrow(predictedRedRows)
    
    # for Blue
    predictedBlueRows <- test[predictedFlags == "Blue", ]
    nCorrectBlues <- sum(predictedBlueRows$flag == 'Blue')
    nPredictedBlues <- nrow(predictedBlueRows)
        
    if (verbose) {
        printf("As for Red: correct/predict = %d/%d = %f\n", nCorrectReds, nPredictedReds, nCorrectReds/nPredictedReds)
        printf("As for Blue: correct/predict = %d/%d = %f\n", nCorrectBlues, nPredictedBlues, nCorrectBlues/nPredictedBlues)  
    }    
    
    c(nCorrectReds/nPredictedReds, nCorrectBlues/nPredictedBlues)
}

In [9]:
# 全体のうち Red と Blue がどれぐらいの割合かを出力する
printRedRatios <- function(dfx) {
    nRed <- nrow(dfx[dfx$flag == "Red", ])
    nAll <- nrow(dfx)
    printf("Red/All = %d/%d = %f\n", nRed, nAll, nRed/nAll)
    printf("1 - Red/All = %d/%d = %f\n", nAll - nRed, nAll, 1 - nRed/nAll)
}

# Predict Reds

In [10]:
df <- read.csv("../data/middle/sp5.csv", stringsAsFactors=FALSE)

In [11]:
df$AccelOrBrake[df$flag == "RedA"] <- "Accel"
df$AccelOrBrake[df$flag == "RedB"] <- "Brake"
df$AccelOrBrake[df$flag == "BlueA"] <- "Accel"
df$AccelOrBrake[df$flag == "BlueB"] <- "Brake"

In [12]:
df$flag[df$flag == "RedA"] <- "Red"
df$flag[df$flag == "RedB"] <- "Red"
df$flag[df$flag == "BlueA"] <- "Blue"
df$flag[df$flag == "BlueB"] <- "Blue"
df$flag <- as.factor(df$flag)

In [13]:
# 存在する RoadType だけ取り出す
roadTypes <- unique(df$RoadType)

In [14]:
# 使用する特徴量だけ抜き取る
df <- df[, c(valuables, "AccelOrBrake", "flag")]

In [15]:
verbose = FALSE

In [16]:
# 各道路タイプに対して CV 値, 長時間平均, 短時間平均を計算
for (i in roadTypes) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df[df$RoadType == i, ]
    result <- CV(dfx, verbose=verbose)
    printRedRatios(dfx)
    printf("Red: Mean correct/predict = %f\n", result[3])
    printf("Blue: Mean correct/predict = %f\n", result[4])
    printf("Short: Mean accel/predictRed = %f\n", result[2]) 
    printf("CV value: %f", result[1])
    printf("\n\n")
}

RoadType: 7 (その他)
Red/All = 25/27 = 0.925926
1 - Red/All = 2/27 = 0.074074
Red: Mean correct/predict = 0.933333
Blue: Mean correct/predict = NaN
Short: Mean accel/predictRed = NaN
CV value: 0.066667

RoadType: 6 (一般道1、一般道2、一般道3)
Red/All = 90/189 = 0.476190
1 - Red/All = 99/189 = 0.523810
Red: Mean correct/predict = 0.674652
Blue: Mean correct/predict = 0.655475
Short: Mean accel/predictRed = 0.591935
CV value: 0.338834

RoadType: 4 (県道)
Red/All = 36/71 = 0.507042
1 - Red/All = 35/71 = 0.492958
Red: Mean correct/predict = 0.532540
Blue: Mean correct/predict = 0.491429
Short: Mean accel/predictRed = 0.680000
CV value: 0.492381

RoadType: 5 (主要地方道)
Red/All = 154/303 = 0.508251
1 - Red/All = 149/303 = 0.491749
Red: Mean correct/predict = 0.618821
Blue: Mean correct/predict = 0.635907
Short: Mean accel/predictRed = 0.693095
CV value: 0.376448

RoadType: 3 (国道)
Red/All = 202/324 = 0.623457
1 - Red/All = 122/324 = 0.376543
Red: Mean correct/predict = 0.673860
Blue: Mean correct/predict = 0.56

# Threshold = 10, 30

In [17]:
# さらに Threshold = 10, 30 で分割する
set.seed(1)

In [18]:
verbose = TRUE

In [25]:
for (i in c(5, 3)) {
    threshold1 <- 10
    threshold2 <- 30
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df[df$RoadType == i, ]
    # dfx を 3 つに分割
    dfx1 <- dfx %>% filter(AverageVelocity <= threshold1)
    dfx2 <- dfx %>% filter(AverageVelocity > threshold1, AverageVelocity <= threshold2)
    dfx3 <- dfx %>% filter(AverageVelocity > threshold2)

    for (j in c(1, 2, 3)) {
        # CV 値, 長時間平均, 短時間平均を計算
        if (j == 1) {
            result <- CV(dfx1, verbose=verbose)        
        } else if (j == 2) {
            result <- CV(dfx2, verbose=verbose)        
        } else {
            result <- CV(dfx3, verbose=verbose)        
        }

        printf("AverageVelocity <= %d:\n", threshold1)
        printf("Red: Mean correct/predict = %f\n", result[3])
        printf("Blue: Mean correct/predict = %f\n", result[4])
        printf("Short: Mean accel/predictRed = %f\n", result[2]) 
        printf("CV value: %f", result[1])        
        printf("\n\n")
    }
}

RoadType: 5 (主要地方道)
Fold1
As for Red: correct/predict = 3/6 = 0.500000
As for Blue: correct/predict = 4/7 = 0.571429
Short: predictedRedAInTrueA/trueRedA = 0/0 = NaN

Fold2
As for Red: correct/predict = 2/2 = 1.000000
As for Blue: correct/predict = 7/12 = 0.583333
Short: predictedRedAInTrueA/trueRedA = 0/0 = NaN

Fold3
As for Red: correct/predict = 6/11 = 0.545455
As for Blue: correct/predict = 3/4 = 0.750000
Short: predictedRedAInTrueA/trueRedA = 0/0 = NaN

Fold4
As for Red: correct/predict = 3/6 = 0.500000
As for Blue: correct/predict = 4/8 = 0.500000
Short: predictedRedAInTrueA/trueRedA = 0/0 = NaN

Fold5
As for Red: correct/predict = 3/6 = 0.500000
As for Blue: correct/predict = 4/7 = 0.571429
Short: predictedRedAInTrueA/trueRedA = 0/0 = NaN

AverageVelocity <= 10:
Red: Mean correct/predict = 0.609091
Blue: Mean correct/predict = 0.595238
Short: Mean accel/predictRed = NaN
CV value: 0.436044

Fold1
As for Red: correct/predict = 16/24 = 0.666667
As for Blue: correct/predict = 10/11 

# Feature Importance

In [21]:
# Random Forest の Feature Importance を計算
df <- df[, c(valuables, "flag")]
for (i in c(0, 2, 3, 4, 5, 6, 7)) {
    dfx <- df[df$RoadType == i, ]
    fit <- randomForest(flag ~ ., data=dfx, mtry=3, ntree=300) 

    # カラム名と importance をカラムとする dataframe を作成
    imps <- data.frame(importance=c(fit$importance), names=valuables)
    printf("RoadType: %d\n", i)
    # importance が高い順にソートして出力
    print(imps[order(imps$importance, decreasing=TRUE), ])
    printf("\n")
}

RoadType: 0
   importance             names
7   2.7821033        RiskFactor
14  2.5853317 AccelerationSpeed
10  2.5551957             Pitch
20  2.3911994      DiffAvgSpeed
17  2.2243369       TimeHeadway
3   2.1832243             Speed
15  2.0717693            Engine
16  2.0650381     SteeringAngle
13  2.0447877   TimeToCollision
11  2.0234981     AheadDistance
12  2.0216457   AverageVelocity
18  1.8940882              Jerk
6   1.6417208          MaxSpeed
5   0.9617544          Curve150
9   0.9566539        DistSignal
21  0.8993679   EmptinessOfRoad
2   0.8578348      CurveAverage
4   0.8155119          Curve100
22  0.5911079        RoadFactor
8   0.5078301             Curve
19  0.4044404         LaneCount
1   0.0000000          RoadType

RoadType: 2
   importance             names
9   2.2831859        DistSignal
10  2.0672820             Pitch
20  1.7572666      DiffAvgSpeed
3   1.6641276             Speed
15  1.1455822            Engine
12  1.1305953   AverageVelocity
14  1.0596805 A

# Feature Importance dvd by RT & VEL

In [22]:
# 更に平均速度で分割したときの Forest の Feature Importance を計算
threshold1 <- 10
threshold2 <- 30

In [23]:
velocities <- c("slow", "middle", "fast")
df <- df[, c(valuables, "flag")]
for (i in c(5, 3)) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df[df$RoadType == i, ]
    # dfx を 3 つに分割
    dfx1 <- dfx %>% filter(AverageVelocity <= threshold1)
    dfx2 <- dfx %>% filter(AverageVelocity > threshold1, AverageVelocity <= threshold2)
    dfx3 <- dfx %>% filter(AverageVelocity > threshold2)
    
    for (j in c(1, 2, 3)) {
        if (j == 1) {
            fit <- randomForest(flag ~ ., data=dfx1, mtry=3, ntree=300) 
        } else if (j == 2) {
            fit <- randomForest(flag ~ ., data=dfx2, mtry=3, ntree=300) 
        } else {
            fit <- randomForest(flag ~ ., data=dfx3, mtry=3, ntree=300) 
        }
        
        # カラム名と importance をカラムとする dataframe を作成
        imps <- data.frame(importance=c(fit$importance), names=valuables)
        printf("RoadType: %d, Velocity: %s\n", i, velocities[j])
        # importance が高い順にソートして出力
        print(imps[order(imps$importance, decreasing=TRUE), ])
        printf("\n")
    }
}

RoadType: 5 (主要地方道)
RoadType: 5, Velocity: slow
   importance             names
17  3.3705413       TimeHeadway
9   2.8056476        DistSignal
16  2.4176145     SteeringAngle
7   2.1065323        RiskFactor
14  2.0742639 AccelerationSpeed
6   2.0636198          MaxSpeed
15  2.0556197            Engine
20  1.9328343      DiffAvgSpeed
18  1.7721771              Jerk
13  1.7587652   TimeToCollision
10  1.7122861             Pitch
12  1.5931214   AverageVelocity
3   1.5011065             Speed
2   1.4235366      CurveAverage
21  1.3948785   EmptinessOfRoad
11  1.2752046     AheadDistance
8   0.7780955             Curve
22  0.4832968        RoadFactor
4   0.4530442          Curve100
19  0.4454194         LaneCount
5   0.4114805          Curve150
1   0.0000000          RoadType

RoadType: 5, Velocity: middle
   importance             names
7    6.807730        RiskFactor
17   6.686917       TimeHeadway
9    6.414286        DistSignal
3    6.330711             Speed
20   5.938333      DiffAv